In [66]:
import pandas as pd
#f = open('./处理过程中的文件/all_data_补充null计数特征.csv')
f = open('./data/all_data_old.csv')
all_data_raw = pd.read_csv(f)

In [67]:
train_y_raw = pd.read_csv('./data/train_y.csv')
train_y = train_y_raw['y']

In [68]:
def split_all_data(all_data_raw):
    train_x_raw = all_data_raw.loc[all_data_raw['belong'] == 1,:]
    train_unlabeled_raw = all_data_raw.loc[all_data_raw['belong'] == 2,:]
    test_x_raw = all_data_raw.loc[all_data_raw['belong'] == 3,:]
    return train_x_raw,train_unlabeled_raw,test_x_raw
#train_x,train_unlabeled,test_x = split_all_data(all_data_raw)

In [70]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn import cross_validation, metrics
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from xgboost.sklearn import XGBClassifier
import numpy as np
import pandas as pd
import os
import shutil
import warnings
warnings.filterwarnings('ignore')

def xgb(all_data,train_y):
    
    
    if os.path.exists('preds_xgb') == True:
        shutil.rmtree('preds_xgb')
    os.mkdir('preds_xgb')
    
    train_x,train_unlabeled,test_x = split_all_data(all_data)
    train_x=train_x.replace(np.nan,-99)
    test_x = test_x.replace(np.nan,-99)
    cust_id = test_x['cust_id']
    auc_score = []
    k = 5
    skf = StratifiedKFold(n_splits=k,shuffle=True,random_state=1)
    feats = [feature for feature in train_x.columns.values if feature not in ['cust_id','cust_group','y','belong']]
    train_x = train_x[feats]
    test_x = test_x[feats]
    print('train_shape',train_x.shape)
    print('test_shape',test_x.shape)
    for k,(train_k,valid_k) in enumerate(skf.split(train_x,train_y)):
        x_train,y_train,x_valid,y_valid = np.array(train_x)[train_k], np.array(train_y)[train_k], np.array(train_x)[valid_k], np.array(train_y)[valid_k]
        print('###################### train!!! ################################')       
        #model_xgb = XGBRegressor(n_estimators=700,max_depth=3,learning_rate=0.07, subsample=0.9,colsample_bytree=0.7)
        model = XGBRegressor(learning_rate=0.01,
                             n_estimators=10000,
                             max_depth=5,
                             min_child_weight=6,
                             gamma=0,
                             subsample=0.7,                  #这个参数控制对于每棵树，随机采样的比例。减小这个参数的值算法会更加保守，避免过拟合。
                                                             #但是这个值设置的过小，它可能会导致欠拟合。典型值：0.5-1 
                             colsample_bytree=0.7,           # 用来控制每颗树随机采样的列数的占比每一列是一个特征0.5-1
                             objective= 'binary:logistic',
                             reg_alpha = 0.01,
                             seed=27)#XGBClassifierXGBRegressor
#         param_test = {#'max_depth':list(range(3,30,3)), 
# #                       'min_child_weight':[list(range(2,10,2))],
#                       'n_estimators': list(range(100,5000,500)),  
#                       'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
#                       'gamma': [i/10.0 for i in range(0,5)],
#                       'subsample':[i/10.0 for i in range(6,10)],
#                       'colsample_bytree':[i/10.0 for i in range(6,10)],
#                       'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
#                      }
        #gsearch = GridSearchCV(estimator = model, param_grid = param_test, scoring='roc_auc', cv=5)
        model.fit(x_train,y_train,\
                eval_set=[(x_valid, y_valid)],\
                eval_metric='auc',\
                early_stopping_rounds=100,\
                verbose = 100)
        #model.fit(x_train,y_train)
        
        print('###################### valid!!! ##################################')
        y_pred_val = model.predict(x_valid)
        auc = roc_auc_score(y_valid,y_pred_val)
        print('The auc_score is',auc)
        auc_score.append(auc)
        print('###################### predict!!! ################################')
        test_pred_y = model.predict(np.array(test_x))
        test_result = pd.DataFrame(columns=["cust_id","pred_prob"])
        test_result.cust_id = cust_id
        test_result.pred_prob = test_pred_y
        test_result.to_csv("./preds_xgb/xgb{0}.csv".format(k),index=None,encoding='utf-8')
    #pred 取平均   
    files = os.listdir('./preds_xgb')
    pred = pd.read_csv('./preds_xgb/'+files[0])
    pred_prob = pred.pred_prob
    for f in files[1:]:
        pred = pd.read_csv('./preds_xgb/'+f)
        pred_prob += pred.pred_prob

    pred_prob /= len(files)

    pred = pd.DataFrame(cust_id,columns=['cust_id']).reset_index(drop = True)
    pred['pred_prob'] = pred_prob
    pred.to_csv('./preds_xgb/avg_preds.csv',index=False)
    return np.mean(auc_score)
xgb(all_data_raw,train_y)




#0.75**3

train_shape (15000, 157)
test_shape (10000, 157)
###################### train!!! ################################
[0]	validation_0-auc:0.720102
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.794419
[200]	validation_0-auc:0.798787
[300]	validation_0-auc:0.803119
[400]	validation_0-auc:0.806547
[500]	validation_0-auc:0.80893
[600]	validation_0-auc:0.810614
Stopping. Best iteration:
[562]	validation_0-auc:0.811092

###################### valid!!! ##################################
The auc_score is 0.8110920068976264
###################### predict!!! ################################
###################### train!!! ################################
[0]	validation_0-auc:0.690017
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.786046


KeyboardInterrupt: 

In [71]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn import cross_validation, metrics
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from xgboost.sklearn import XGBClassifier
import numpy as np
import pandas as pd
import os
import shutil
import warnings
warnings.filterwarnings('ignore')

def xgb(all_data,train_y):
    
    
    if os.path.exists('preds_xgb') == True:
        shutil.rmtree('preds_xgb')
    os.mkdir('preds_xgb')
    
    train_x,train_unlabeled,test_x = split_all_data(all_data)
    train_x=train_x.replace(np.nan,-99)
    test_x = test_x.replace(np.nan,-99)
    cust_id = test_x['cust_id']
    auc_score = []
    k = 5
    skf = StratifiedKFold(n_splits=k,shuffle=True,random_state=1)
    feats = [feature for feature in train_x.columns.values if feature not in ['cust_id','cust_group','y','belong']]
    train_x = train_x[feats]
    test_x = test_x[feats]
    print('train_shape',train_x.shape)
    print('test_shape',test_x.shape)
    for k,(train_k,valid_k) in enumerate(skf.split(train_x,train_y)):
        x_train,y_train,x_valid,y_valid = np.array(train_x)[train_k], np.array(train_y)[train_k], np.array(train_x)[valid_k], np.array(train_y)[valid_k]
        print('###################### train!!! ################################')       
        #model_xgb = XGBRegressor(n_estimators=700,max_depth=3,learning_rate=0.07, subsample=0.9,colsample_bytree=0.7)
        model = XGBRegressor(learning_rate=0.01,
                             n_estimators=10000,
                             max_depth=5,
                             min_child_weight=6,
                             gamma=0,
                             subsample=0.6,                  #这个参数控制对于每棵树，随机采样的比例。减小这个参数的值算法会更加保守，避免过拟合。
                                                             #但是这个值设置的过小，它可能会导致欠拟合。典型值：0.5-1 
                             colsample_bytree=0.6,           # 用来控制每颗树随机采样的列数的占比每一列是一个特征0.5-1
                             objective= 'binary:logistic',
                             reg_alpha = 0.01,
                             seed=27)#XGBClassifierXGBRegressor
#         param_test = {#'max_depth':list(range(3,30,3)), 
# #                       'min_child_weight':[list(range(2,10,2))],
#                       'n_estimators': list(range(100,5000,500)),  
#                       'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
#                       'gamma': [i/10.0 for i in range(0,5)],
#                       'subsample':[i/10.0 for i in range(6,10)],
#                       'colsample_bytree':[i/10.0 for i in range(6,10)],
#                       'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
#                      }
        #gsearch = GridSearchCV(estimator = model, param_grid = param_test, scoring='roc_auc', cv=5)
        model.fit(x_train,y_train,\
                eval_set=[(x_valid, y_valid)],\
                eval_metric='auc',\
                early_stopping_rounds=100,\
                verbose = 100)
        #model.fit(x_train,y_train)
        
        print('###################### valid!!! ##################################')
        y_pred_val = model.predict(x_valid)
        auc = roc_auc_score(y_valid,y_pred_val)
        print('The auc_score is',auc)
        auc_score.append(auc)
        print('###################### predict!!! ################################')
        test_pred_y = model.predict(np.array(test_x))
        test_result = pd.DataFrame(columns=["cust_id","pred_prob"])
        test_result.cust_id = cust_id
        test_result.pred_prob = test_pred_y
        test_result.to_csv("./preds_xgb/xgb{0}.csv".format(k),index=None,encoding='utf-8')
    #pred 取平均   
    files = os.listdir('./preds_xgb')
    pred = pd.read_csv('./preds_xgb/'+files[0])
    pred_prob = pred.pred_prob
    for f in files[1:]:
        pred = pd.read_csv('./preds_xgb/'+f)
        pred_prob += pred.pred_prob

    pred_prob /= len(files)

    pred = pd.DataFrame(cust_id,columns=['cust_id']).reset_index(drop = True)
    pred['pred_prob'] = pred_prob
    pred.to_csv('./preds_xgb/avg_preds.csv',index=False)
    return np.mean(auc_score)
xgb(all_data_raw,train_y)


train_shape (15000, 157)
test_shape (10000, 157)
###################### train!!! ################################
[0]	validation_0-auc:0.720102
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.794419
[200]	validation_0-auc:0.798787
[300]	validation_0-auc:0.803119
[400]	validation_0-auc:0.806547
[500]	validation_0-auc:0.80893
[600]	validation_0-auc:0.810614
Stopping. Best iteration:
[562]	validation_0-auc:0.811092

###################### valid!!! ##################################
The auc_score is 0.8110920068976264
###################### predict!!! ################################
###################### train!!! ################################
[0]	validation_0-auc:0.690017
Will train until validation_0-auc hasn't improved in 100 rounds.
[100]	validation_0-auc:0.786046
[200]	validation_0-auc:0.794768
[300]	validation_0-auc:0.802211
[400]	validation_0-auc:0.805401
[500]	validation_0-auc:0.808161
[600]	validation_0-auc:0.807746
Stopping. Best iter

0.8193171488465312

In [54]:
param_test1 = {
 'max_depth':[1,2,3],
 'min_child_weight':[1,2,3]
}
train_x,train_unlabeled,test_x = split_all_data(all_data_raw)
train_x=train_x.replace(np.nan,-99)
test_x = test_x.replace(np.nan,-99)
feats = [feature for feature in train_x.columns.values if feature not in ['cust_id','cust_group','y','belong']]
train_x = train_x[feats]
test_x = test_x[feats]
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=140, 
                                                  #max_depth=5,
                                                  #min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8,
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', 
                                                  nthread=4,
                                                  scale_pos_weight=1,
                                                  seed=27), 
                                                  param_grid = param_test1,
                                                  scoring='roc_auc',
                                                  n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_x,train_y)
gsearch1.grid_scores_, gsearch1.best_params_,     gsearch1.best_score_

([mean: 0.80881, std: 0.04881, params: {'max_depth': 1, 'min_child_weight': 1},
  mean: 0.80923, std: 0.04856, params: {'max_depth': 1, 'min_child_weight': 2},
  mean: 0.80885, std: 0.04825, params: {'max_depth': 1, 'min_child_weight': 3},
  mean: 0.81736, std: 0.04535, params: {'max_depth': 2, 'min_child_weight': 1},
  mean: 0.81621, std: 0.04483, params: {'max_depth': 2, 'min_child_weight': 2},
  mean: 0.81540, std: 0.04587, params: {'max_depth': 2, 'min_child_weight': 3},
  mean: 0.81510, std: 0.04394, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.81408, std: 0.04403, params: {'max_depth': 3, 'min_child_weight': 2},
  mean: 0.81567, std: 0.04442, params: {'max_depth': 3, 'min_child_weight': 3}],
 {'max_depth': 2, 'min_child_weight': 1},
 0.81736234004312)